In [5]:
import torch
import torch.nn.functional as F
print(torch.__version__)

2.4.0.dev20240507


In [6]:
from executorch.examples.models.llama2.builder import load_llama_model, DType
from executorch.examples.models.llama2.source_transformation.quantize import WeightOnlyInt8QuantHandler
from executorch.examples.models.llama2.source_transformation.sdpa import replace_sdpa_with_simple_sdpa, replace_sdpa_with_custom_op

In [7]:
# stories110M

checkpoint = "stories110M.pt"
params = "params_110M.json"
transforms = [
    lambda m: WeightOnlyInt8QuantHandler(m).quantized_model(),
    replace_sdpa_with_custom_op,
]

model = load_llama_model(
    checkpoint=checkpoint,
    params_path=params,
    use_kv_cache=True,
    use_sdpa_with_kv_cache=True
).to_dtype(DType.fp32).source_transform(transforms)

[INFO 2024-06-07 13:54:55,708 builder.py:84] Loading model with checkpoint=/Users/larryliu/Meta-Llama-3-8B/consolidated.00.pth, params=/Users/larryliu/Meta-Llama-3-8B/config.json, use_kv_cache=True, weight_type=WeightType.LLAMA


RuntimeError: PytorchStreamReader failed reading zip archive: failed finding central directory

In [4]:
print(model.model.layers[0].attention.n_local_kv_heads)

6


In [4]:
from executorch.backends.apple.mps.partition.mps_partitioner import (
    MPSPartitioner,
)
from executorch.exir.backend.backend_details import CompileSpec
compile_specs = [CompileSpec("use_fp16", bytes([True]))]

partitioners = [
    MPSPartitioner(compile_specs)
]

## Pass to replace aten::_weight_int8pack_mm to llama_cpp::_weight_int8pack_mm

In [11]:
from executorch.exir.pass_base import ExportPass
from executorch.examples.models.llama2.custom_ops.llama_cpp_linear import *
from executorch.exir.dialects._ops import ops as exir_ops

class ReplaceMMPass(ExportPass):

    def __init__(self):
        super().__init__()

    def call_operator(self, op, args, kwargs, meta):
        if op == exir_ops.edge.aten._weight_int8pack_mm.default:
            return super().call_operator(exir_ops.edge.llama_cpp._weight_int8pack_mm.default, args, kwargs, meta)
        else:
            return super().call_operator(op, args, kwargs, meta)



In [12]:
builder = model.export_to_edge()
builder.edge_manager = builder.edge_manager.transform([ReplaceMMPass()])
builder.to_backend(partitioners).to_executorch()

[INFO 2024-05-31 16:58:16,522 mps_partitioner.py:121] Found 7 subgraphs to be partitioned.
[INFO 2024-05-31 16:58:16,578 mps_preprocess.py:115] Visiting: aten_view_copy_default_49, aten.view_copy.default
[INFO 2024-05-31 16:58:16,578 mps_preprocess.py:115] Visiting: aten_view_copy_default_55, aten.view_copy.default
[INFO 2024-05-31 16:58:16,579 mps_preprocess.py:115] Visiting: aten_view_copy_default_56, aten.view_copy.default
[INFO 2024-05-31 16:58:16,579 mps_preprocess.py:115] Visiting: aten__to_copy_default_31, aten._to_copy.default
[INFO 2024-05-31 16:58:16,580 mps_preprocess.py:115] Visiting: aten__to_copy_default_32, aten._to_copy.default
[INFO 2024-05-31 16:58:16,580 mps_preprocess.py:115] Visiting: aten__to_copy_default_33, aten._to_copy.default
[INFO 2024-05-31 16:58:16,580 mps_preprocess.py:115] Visiting: aten__to_copy_default_34, aten._to_copy.default
[INFO 2024-05-31 16:58:16,580 mps_preprocess.py:115] Visiting: aten__to_copy_default_35, aten._to_copy.default
[INFO 2024-05-3

In [6]:
from executorch.examples.models.llama2.lib.partitioner_lib import get_xnnpack_partitioner
builder = model.export_to_edge()
builder.edge_manager = builder.edge_manager.transform([ReplaceMMPass()])
builder.to_backend([get_xnnpack_partitioner()]).to_executorch().save_to_pte("stories15M_int8_xnnpack_llama_cpp.pte")

[WARNING 2024-05-31 12:55:46,709 xnnpack_partitioner.py:558] Nothing can be partitioned!
/Users/larryliu/miniconda3/envs/executorch/lib/python3.11/site-packages/executorch/exir/emit/_emitter.py:1474: UserWarning: Mutation on a buffer in the model is detected. ExecuTorch assumes buffers that are mutated in the graph have a meaningless initial state, only the shape and dtype will be serialized.
  warnings.warn(
[INFO 2024-05-31 12:55:47,565 builder.py:375] Required memory for activation in bytes: [0, 1923776]
[INFO 2024-05-31 12:55:47,580 utils.py:114] Saved exported program to stories15M_int8_xnnpack_llama_cpp.pte


In [3]:
builder = model.export_to_edge(None).to_executorch()

/Users/larryliu/miniconda3/envs/executorch/lib/python3.11/site-packages/executorch/exir/emit/_emitter.py:1474: UserWarning: Mutation on a buffer in the model is detected. ExecuTorch assumes buffers that are mutated in the graph have a meaningless initial state, only the shape and dtype will be serialized.
  warnings.warn(
[INFO 2024-05-08 18:05:25,534 builder.py:341] Required memory for activation in bytes: [0, 418116608]


In [ ]:
print(model.edge_manager._edge_programs['forward'].graph)

In [13]:
from executorch.exir.backend.utils import print_delegated_graph

print_delegated_graph(builder.export_program.exported_program("forward").graph_module)

graph():
  %b_layers_0_attention_sdpa_kv_cache_k_cache : [num_users=1] = placeholder[target=b_layers_0_attention_sdpa_kv_cache_k_cache]
  %b_layers_0_attention_sdpa_kv_cache_v_cache : [num_users=1] = placeholder[target=b_layers_0_attention_sdpa_kv_cache_v_cache]
  %b_layers_1_attention_sdpa_kv_cache_k_cache : [num_users=1] = placeholder[target=b_layers_1_attention_sdpa_kv_cache_k_cache]
  %b_layers_1_attention_sdpa_kv_cache_v_cache : [num_users=1] = placeholder[target=b_layers_1_attention_sdpa_kv_cache_v_cache]
  %b_layers_2_attention_sdpa_kv_cache_k_cache : [num_users=1] = placeholder[target=b_layers_2_attention_sdpa_kv_cache_k_cache]
  %b_layers_2_attention_sdpa_kv_cache_v_cache : [num_users=1] = placeholder[target=b_layers_2_attention_sdpa_kv_cache_v_cache]
  %b_layers_3_attention_sdpa_kv_cache_k_cache : [num_users=1] = placeholder[target=b_layers_3_attention_sdpa_kv_cache_k_cache]
  %b_layers_3_attention_sdpa_kv_cache_v_cache : [num_users=1] = placeholder[target=b_layers_3_attentio

In [20]:
builder.save_to_pte("stories15M_int8_mps_llama_cpp.pte")

[INFO 2024-05-31 13:39:36,765 utils.py:114] Saved exported program to stories15M_int8_mps_llama_cpp.pte


In [14]:
builder.save_to_pte("stories15M_int8_mps_sdpa.pte")

[INFO 2024-05-31 17:00:06,644 utils.py:114] Saved exported program to stories15M_int8_mps_sdpa.pte


In [16]:
builder.save_to_pte("stories15M_int8_llama_cpp.pte")

[INFO 2024-05-31 13:32:31,174 utils.py:114] Saved exported program to stories15M_int8_llama_cpp.pte


In [20]:
from executorch.extension.pybindings.portable_lib import _get_operator_names

names = _get_operator_names()

In [23]:
print('\n'.join(names))
print(len(names))

aten::sym_size.int
aten::_local_scalar_dense
aten::sym_numel
executorch_prim::add.Scalar
executorch_prim::sub.Scalar
executorch_prim::mul.Scalar
executorch_prim::floordiv.Scalar
executorch_prim::truediv.Scalar
executorch_prim::eq.Scalar
executorch_prim::gt.Scalar
executorch_prim::lt.Scalar
executorch_prim::ge.Scalar
executorch_prim::le.Scalar
executorch_prim::floordiv.int
executorch_prim::et_copy_index.tensor
executorch_prim::et_view.default
aten::_cdist_forward.out
aten::_log_softmax.out
aten::_native_batch_norm_legit.out
aten::_native_batch_norm_legit.no_stats_out
aten::_native_batch_norm_legit_no_training.out
aten::_pdist_forward.out
aten::_softmax.out
aten::_to_copy.out
aten::abs.out
aten::acos.out
aten::acosh.out
aten::add.out
aten::add.Scalar_out
aten::addmm.out
aten::alias_copy.out
aten::amax.out
aten::amin.out
aten::any.all_out
aten::any.dims_out
aten::any.out
aten::arange.out
aten::arange.start_out
aten::argmax.out
aten::argmin.out
aten::as_strided_copy.out
aten::asin.out
aten

In [6]:
builder.save_to_pte("stories110M_int8.pte")

[INFO 2024-05-08 14:39:40,251 utils.py:112] Saved exported program to stories110M_int8.pte


In [7]:
builder.save_to_pte("stories110M_int8_mps.pte")

[INFO 2024-05-08 17:58:18,813 utils.py:113] Saved exported program to stories110M_int8_mps.pte


In [7]:
from executorch.extension.pybindings.portable_lib import _load_for_executorch

In [56]:
op1 = torch.ops.aten._weight_int8pack_mm.default
op2 = torch.ops.llama_cpp._weight_int8pack_mm.default
mps_device = torch.device("mps")  # Device object representing GPU.

A = torch.randn(4, 8, dtype=torch.float, device=mps_device)
B = torch.ones(8, 8, dtype=torch.int8, device=mps_device)
scales = torch.randn(8, dtype=torch.float, device=mps_device)


In [57]:
C1 = op1(A, B, scales)
print(C1)

tensor([[ 0.2965,  0.1211, -0.0488, -0.0738, -0.5035,  0.7861,  0.9290,  0.6385],
        [ 2.7348,  1.1167, -0.4501, -0.6806, -4.6442,  7.2503,  8.5685,  5.8890],
        [-0.0908, -0.0371,  0.0149,  0.0226,  0.1542, -0.2408, -0.2845, -0.1956],
        [-2.2508, -0.9191,  0.3705,  0.5601,  3.8223, -5.9672, -7.0521, -4.8468]],
       device='mps:0')


In [58]:
C2 = op2(A, B, scales)
print(C2)

tensor([[ 0.2965,  0.1211, -0.0488, -0.0738, -0.5035,  0.7861,  0.9290,  0.6385],
        [ 2.7348,  1.1167, -0.4501, -0.6806, -4.6442,  7.2503,  8.5685,  5.8890],
        [-0.0908, -0.0371,  0.0149,  0.0226,  0.1542, -0.2408, -0.2845, -0.1956],
        [-2.2508, -0.9191,  0.3705,  0.5601,  3.8223, -5.9672, -7.0521, -4.8468]],
       device='mps:0')


In [59]:
torch.allclose(C1, C2)

True

In [63]:
from sentencepiece import SentencePieceProcessor as SPP

sp_model = SPP(model_file="tokenizer.model")

In [64]:
prompt = "Once upon a time"

prompt_tokens = sp_model.encode(prompt)
prompt_tokens = [sp_model.bos_id()] + prompt_tokens
print(prompt_tokens)

[1, 9038, 2501, 263, 931]


In [65]:
t = torch.tensor([[prompt_tokens[0]]], dtype=torch.int64)
pos = torch.tensor([0], dtype=torch.int64)

model.model(t, pos)

: 

: 

In [13]:
# edge module
builder = model.export_to_edge()
edge_module = builder.edge_manager._edge_programs["forward"].module()

In [14]:
# custom op module
builder.edge_manager = builder.edge_manager.transform([ReplaceMMPass()])
custom_op_module = builder.edge_manager._edge_programs["forward"].module()

In [13]:
# try xnnpack
from executorch.examples.models.llama2.lib.partitioner_lib import get_xnnpack_partitioner
builder.to_backend([get_xnnpack_partitioner()]).to_executorch()

from executorch.extension.pybindings._portable_lib import _load_for_executorch_from_buffer

ep = _load_for_executorch_from_buffer(builder.export_program.buffer)

[WARNING 2024-05-29 15:00:15,232 xnnpack_partitioner.py:558] Nothing can be partitioned!
/Users/larryliu/miniconda3/envs/executorch/lib/python3.11/site-packages/executorch/exir/emit/_emitter.py:1474: UserWarning: Mutation on a buffer in the model is detected. ExecuTorch assumes buffers that are mutated in the graph have a meaningless initial state, only the shape and dtype will be serialized.
  warnings.warn(
[INFO 2024-05-29 15:00:17,022 builder.py:375] Required memory for activation in bytes: [0, 4587264]
[program.cpp:130] InternalConsistency verification requested but not available


In [17]:
# lowered to mps module
builder.to_backend(partitioners)
mps_module = builder.edge_manager._edge_programs["forward"].module()

[INFO 2024-05-31 15:10:47,125 mps_partitioner.py:121] Found 31 subgraphs to be partitioned.
[INFO 2024-05-31 15:10:47,158 mps_preprocess.py:115] Visiting: aten_sigmoid_default_5, aten.sigmoid.default
[INFO 2024-05-31 15:10:47,159 mps_preprocess.py:115] Visiting: aten_mul_tensor_94, aten.mul.Tensor
[INFO 2024-05-31 15:10:47,159 mps_preprocess.py:115] Visiting: aten_mul_tensor_95, aten.mul.Tensor
[INFO 2024-05-31 15:10:47,301 mps_preprocess.py:115] Visiting: aten_add_tensor_33, aten.add.Tensor
[INFO 2024-05-31 15:10:47,302 mps_preprocess.py:115] Visiting: aten_mul_tensor_91, aten.mul.Tensor
[INFO 2024-05-31 15:10:47,302 mps_preprocess.py:115] Visiting: aten_mean_dim_11, aten.mean.dim
[INFO 2024-05-31 15:10:47,303 mps_preprocess.py:115] Visiting: aten_add_tensor_34, aten.add.Tensor
[INFO 2024-05-31 15:10:47,303 mps_preprocess.py:115] Visiting: aten_rsqrt_default_11, aten.rsqrt.default
[INFO 2024-05-31 15:10:47,303 mps_preprocess.py:115] Visiting: aten_mul_tensor_92, aten.mul.Tensor
[INFO 

In [56]:
# to executorch
builder.to_executorch()
from executorch.extension.pybindings._portable_lib import _load_for_executorch_from_buffer

ep = _load_for_executorch_from_buffer(builder.export_program.buffer)

[program.cpp:130] InternalConsistency verification requested but not available
loc("mps_broadcast_to"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/91a344b1-f985-11ee-b563-fe8bc7981bff/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":1392:0)): error: 'anec.broadcast' op failed: input cannot be broadcasted to the target shape


loc("mps_broadcast_to"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/91a344b1-f985-11ee-b563-fe8bc7981bff/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":1392:0)): error: 'anec.broadcast' op failed: input cannot be broadcasted to the target shape


In [30]:
# eager result
print(model.model.forward(t, pos))

tensor([[-5.8514,  1.4318, -5.8515,  ..., -5.8277, -5.8277, -5.8515]],
       grad_fn=<NotImplemented>)


In [28]:
# edge result
print(edge_module.forward(t, pos))

tensor([[-5.8514,  1.4318, -5.8515,  ..., -5.8277, -5.8277, -5.8515]],
       grad_fn=<NotImplemented>)


In [31]:
# edge with custom op
print(custom_op_module.forward(t, pos))

tensor([[-5.8514,  1.4318, -5.8515,  ..., -5.8277, -5.8277, -5.8515]],
       grad_fn=<NotImplemented>)


In [32]:
# mps result
print(mps_module.forward(t, pos))

tensor([[-5.8514,  1.4318, -5.8515,  ..., -5.8277, -5.8277, -5.8515]],
       grad_fn=<NotImplemented>)


In [14]:
#ep result
print(ep.forward((t, pos)))

[tensor([[-5.8514,  1.4318, -5.8515,  ..., -5.8276, -5.8277, -5.8515]])]


In [67]:
print(builder.export_program)

In [15]:
builder.save_to_pte("stories15M_int8_xnnpack_llama_cpp.pte")

[INFO 2024-05-29 15:02:07,059 utils.py:114] Saved exported program to stories15M_int8_xnnpack_llama_cpp.pte


In [22]:
print(builder.export_program._emitter_output.program.execution_plan[0].chains[0].instructions[35])

Instruction(instr_args=KernelCall(op_index=2, args=[221, 235, 245, 245]))


In [26]:
mps_module = builder.export_program.exported_program("forward").graph_module

In [30]:
import torch
from executorch.exir.delegate import executorch_call_delegate


In [34]:
for node in mps_module.graph.nodes:
    if node.target == executorch_call_delegate:
        print(node.args)


(lowered_module_0, tokens)
(lowered_module_1, llama_cpp__weight_int8pack_mm_default, llama_cpp__weight_int8pack_mm_default_1, llama_cpp__weight_int8pack_mm_default_2, input_pos)
(lowered_module_2, sdpa_with_kv_cache)
(lowered_module_3, getitem, llama_cpp__weight_int8pack_mm_default_3)
(lowered_module_4, llama_cpp__weight_int8pack_mm_default_4, llama_cpp__weight_int8pack_mm_default_5)
(lowered_module_5, getitem_10, llama_cpp__weight_int8pack_mm_default_6)
(lowered_module_6, getitem_5, getitem_6, llama_cpp__weight_int8pack_mm_default_7, llama_cpp__weight_int8pack_mm_default_8, llama_cpp__weight_int8pack_mm_default_9)
(lowered_module_7, sdpa_with_kv_cache_1)
(lowered_module_8, getitem_14, llama_cpp__weight_int8pack_mm_default_10)
(lowered_module_9, llama_cpp__weight_int8pack_mm_default_11, llama_cpp__weight_int8pack_mm_default_12)
(lowered_module_10, getitem_22, llama_cpp__weight_int8pack_mm_default_13)
(lowered_module_11, getitem_5, getitem_6, llama_cpp__weight_int8pack_mm_default_14, ll